## Others 🍄

---

### Dependencies:

In [2]:
import yaml
import re
from datetime import datetime
import pandas as pd

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [23]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

In [24]:
edge:object = webdriver.Edge(executable_path="./utils/msedgedriver.exe")

### Functions:

In [25]:
def resources(name:str) -> dict[str:str]:
    """Read xpath.yml file that contains xpath's to scrape"""

    with open("xpath.yml", "r") as f:
        content:dict[str:str] = yaml.safe_load(f)
        return content[name]

In [26]:
def close_prompt(driver:object) -> None:
    """Close ad window"""

    button:str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [27]:
def choose_category(category:str, driver:object) -> None:
    """Choose ad category"""

    driver.find_element(by=By.XPATH, value=category).click()

In [28]:
def click_element(element:object, path:str) -> None:
    """Click element on the current document"""

    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [29]:
def extract_description(element:object, path:str) -> str:
    """Extract ad description on the current document"""

    description:str = element.find_element(by=By.XPATH, value=path)
    return description

In [30]:
def extract_feature(element:object, path:str) -> list[str]:
    """Extract some feature from list document based on xpath used."""

    feature = []

    try:
        feature:str = element.find_elements(by=By.XPATH, value=path)
        feature = [data_point.text for data_point in feature]
        return feature
    except Exception:
        feature.append("Web wasn't able to scrape the feature")

In [31]:
def persistence(dataframe:pd.DataFrame) -> None:
    """Generate .csv file from data scraped"""

    file_name = datetime.today().strftime("%B-%d")
    dataframe.to_csv(f"persistence/{file_name}.csv", index=False)

In [32]:
def parse_price(dataframe: pd.DataFrame, column:str, pattern:str) -> None:
    """
    Cast string prices to int prices.
    Works with $#,### or $#,###\n* pattern
    """


    dataframe[column] = dataframe [column] \
        .str.extract(pattern) \
        .applymap(lambda price: int(price.replace(",","")))


In [55]:
def check_len(features:list[list[str]]) -> bool:
    base_len = len(features[0])
    lengths = []
    lengths.append(base_len)
    for feature in features:
        if len(feature) == base_len:
            continue
        else:
             lengths.append(len(feature))
    return lengths

In [22]:
def remove_chars(dataframe: pd.DataFrame, feature:str, wildcards:str) -> None:
    return (dataframe
            .apply(
            lambda row: dataframe[feature]
            ,axis = 1
    )
            .apply(
            lambda extracted_f: re.sub(wildcards, r' ', str(extracted_f))
        )
)

## Extraction 🧲

----

### Sedan:

Variables pool.

In [33]:
descriptions:list[str] = []
prices:list[int] = []
models:list[str] = []
years:list[int] = []
kilometers:list[int] = []
engines:list[str] = []

# seconds
DELAY:int = 3

In [34]:
sedan = resources("cars_sedan")

Get web & close initial prompt.

In [35]:
edge.get(sedan["url"])

In [36]:
close_prompt(edge)

In [37]:
elements = edge.find_elements(by=By.XPATH, value=sedan["ad"])

Extract descriptions.

In [38]:
elements[0].click()
try:
    for element in range(len(elements)):
        dynamic_delay = WebDriverWait(edge, DELAY).until(EC.presence_of_element_located((By.XPATH, sedan["description"])))
        temp = extract_description(edge, sedan["description"])
        descriptions.append(temp.text)
        click_element(edge, sedan["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    click_element(edge, sedan["exit"])

Extract prices & models.

In [39]:
prices = extract_feature(edge, sedan["price"])
models = extract_feature(edge, sedan["model"])
# models.remove("Blue Book Encuentra24")

Extract car year, engine & kilometers.

In [40]:
years = extract_feature(edge, sedan["year"])
engines = extract_feature(edge, sedan["engine"])
kilometers = extract_feature(edge, sedan["kilometers"])

In [56]:
lengths = [prices, models, years, engines, kilometers, descriptions]
check_len(lengths)

[47, 48, 48, 46]

## Transformation ⚙

There exist a missing kilometer value, so in kilometers actually was append a engine type instead a kilometer value due it's value is missing.

---

In [ ]:
# years.remove("Ver más")
# engines.append("Gasolina")
# kilometers.remove("Gasolina")
# kilometers.insert(-2,"0")

Temporal df to apply transformation and after save it as csv.

In [41]:
df_temp = pd.DataFrame({
    "model": models,
    "year": years,
    "kilometers": kilometers,
    "engine": engines,
    "price": prices,
    "description": descriptions
})

ValueError: All arrays must be of the same length

### Parse columns:

Chance column dtype from str -> int

In [ ]:
price_rg:str = r"^\$(?P<prices>\d+,\d+)?"
parse_price(df_temp, "price", price_rg)

In [ ]:
km_rg: str = r"(?P<km>^\d+)"
df_temp["kilometers"] = df_temp["kilometers"].str.extract(km_rg).applymap(int)

In [ ]:
df_temp["year"] = df_temp["year"].map(int)

In [209]:
persistence(df_temp)

### Dummies:

Dummies for categorical variables

In [4]:
df = pd.read_csv("./persistence/February-13.csv")
df_dum = df.copy()

In [17]:
dummies = pd.get_dummies(df_dum["engine"], prefix="engine")
df_dum = pd.concat([df_dum, dummies], axis=1)

In [20]:
df_dum = df_dum.drop(["engine"], axis=1)

### Stats:

Basic information about new extraction stored in df.